In [1]:
import pandas as pd


In [2]:
import sqlite3
from sqlite3 import Error

# Added:
database = 'sqlite_db_pythonsqlite.db'
con = sqlite3.connect(database)
#####################################################

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [4]:
# Testing
pd.read_sql_query('SELECT * FROM Facilities', con)


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [8]:
#Q10
'''Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! ''' 


q10 = '''
SELECT f.name AS facility,
		SUM(CASE WHEN b.memid > 0 THEN b.slots * f.membercost
                 ELSE b.slots * f.guestcost END) AS revenue
FROM Bookings as b
	INNER JOIN Members as m
	ON b.memid = m.memid
	INNER JOIN Facilities as f
	ON b.facid = f.facid
GROUP BY facility
HAVING revenue < 1000
ORDER BY revenue
'''
pd.read_sql_query(q10, con)

,facility,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [31]:
'''/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */'''

q11 = '''
SELECT t1.surname AS mem_surname, t1.firstname AS mem_firstname, (m3.surname || '  ' || m3.firstname) as Recommender
FROM
    (SELECT m1.memid, m1.surname, m1.firstname, m1.recommendedby
    FROM Members as m1
    INNER JOIN Members as m2
    ON m1.recommendedby = m2.memid) AS t1
        INNER JOIN Members as m3
            ON t1.recommendedby = m3.memid
ORDER BY mem_surname, mem_firstname
'''

pd.read_sql_query(q11, con)

,mem_surname,mem_firstname,Recommender
0,Bader,Florence,Stibbons Ponder
1,Baker,Anne,Stibbons Ponder
2,Baker,Timothy,Farrell Jemima
3,Boothe,Tim,Rownam Tim
4,Butters,Gerald,Smith Darren
5,Coplin,Joan,Baker Timothy
6,Crumpet,Erica,Smith Tracy
7,Dare,Nancy,Joplette Janice
8,Genting,Matthew,Butters Gerald
9,Hunt,John,Purview Millicent


In [74]:
'''/* Q12: Find the facilities with their usage by member, but not guests */'''

q12 = '''
SELECT f.name AS facility, (m.surname || '  ' || m.firstname) AS member, SUM(slots) as tot_usage
FROM Facilities AS f
LEFT JOIN Bookings AS b
    ON f.facid = b.facid
LEFT JOIN Members AS m
    ON b.memid = m.memid
WHERE b.memid > 0
GROUP BY facility, member
'''

pd.read_sql_query(q12, con)

,facility,member,tot_usage
0,Badminton Court,Bader Florence,27
1,Badminton Court,Baker Anne,30
2,Badminton Court,Baker Timothy,21
3,Badminton Court,Boothe Tim,36
4,Badminton Court,Butters Gerald,63
...,...,...,...
197,Tennis Court 2,Smith Darren,57
198,Tennis Court 2,Smith Jack,3
199,Tennis Court 2,Smith Tracy,6
200,Tennis Court 2,Stibbons Ponder,96


In [72]:
'''/* Q13: Find the facilities usage by month, but not guests */'''
q13 = '''
SELECT f.name AS facility, SUM(slots) AS tot_usage, strftime('%m', date(starttime)) AS Month
FROM Bookings as b
LEFT JOIN Facilities as f
    ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY strftime('%m', date(starttime)), facility
'''

pd.read_sql_query(q13, con)

,facility,tot_usage,Month
0,Badminton Court,165,07
1,Massage Room 1,166,07
2,Massage Room 2,8,07
3,Pool Table,110,07
4,Snooker Table,140,07
5,Squash Court,50,07
6,Table Tennis,98,07
7,Tennis Court 1,201,07
8,Tennis Court 2,123,07
9,Badminton Court,414,08
